In [1]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

In [2]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math

In [4]:
data = get_input(11, 2017)[0].split(',')

def move(pos, dir):
  x,y = pos
  if dir == 'n':
    return (x, y-2)
  elif dir == 's':
    return (x, y+2)
  elif dir == 'ne':
    return (x+1,y-1)
  elif dir == 'se':
    return (x+1,y+1)
  elif dir == 'sw':
    return (x-1,y+1)
  elif dir == 'nw':
    return (x-1,y-1)

def moves(pos, dirs):
  for dir in dirs:
    pos = move(pos, dir)
  return pos

assert moves((0,0), ['n','s']) == (0,0)
assert moves((0,0), ['ne','sw']) == (0,0)
assert moves((0,0), ['ne','nw', 's']) == (0,0)
assert moves((0,0), ['ne','nw', 's']) == (0,0)
assert moves((0,0), ['se','n', 'nw', 's']) == (0,0)
assert moves((0,0), ['sw','n', 'se']) == (0,0)
assert moves((0,0), ['sw','se', 'n']) == (0,0)

def hex_dist(a,b):
  x,y = a
  x2,y2 = b
  dy = abs(y2-y)
  dx = abs(x2-x)
  steps = 0

  # go diag
  while dx >= 1 and dy >= 1:
    dx -= 1
    dy -= 1
    steps += 1

  if dx == 0 and dy == 0: return steps

  while not (dx == 0 and dy == 0):
    if dx == 0:
      assert dy >= 2
      assert dy % 2 == 0
      steps += dy // 2
      return steps
    if dy == 0:
      steps += dx
      return steps
    assert False
  assert False

pos = moves( (0,0), data)
pos

p1 = hex_dist((0,0), pos)


max_steps = 0
pos = (0,0)
for line in data:
  pos = move(pos, line)
  steps = hex_dist((0,0), pos)
  if steps > max_steps:
    max_steps = steps

p2 = max_steps
p1,p2


(707, 1490)

In [4]:
data = get_input(12, 2017)
G = defaultdict(set)
for line in data:
  src,*targets = aoc_utils.mapints(line)
  G[src] |= set(targets)

def find_group(start):
  seen = set()
  search = [start]
  while len(search):
    cur = search.pop()
    seen.add(cur)
    for t in G[cur]:
      if t in seen: continue
      search.append(t)
  return seen

p1 = len(find_group(0))
p1

groups = []
for src in G:
  if any(src in g for g in groups): continue
  groups.append(find_group(src))
p2 = len(groups)

p1,p2



(113, 202)

In [ ]:
data = get_input(13, 2017)
testdata ="""0: 3
1: 2
4: 4
6: 4""".split("\n")

from functools import cache

@cache
def calc(rng, time):
  arr = get_rng(rng)
  return arr[time % len(arr)]

@cache
def get_rng(rng):
  from itertools import chain
  arr = list(chain(range(rng), reversed(range(1,rng-1))))
  return arr


def run(delay=0):
  tot = 0
  caught = 0
  for l in data:
    depth,rng = aoc_utils.mapints(l)
    v = calc(rng, depth+delay)
    if v == 0:
      tot += depth*rng
      caught += 1
  return tot,caught

p1,_ = run(delay=0)

p2 = -1
for i in range(3_000_000,10_000_000):
  if run(i)[1] == 0:
    p2 = i
    break

# 34124 is too low
p1,p2

(1612, 3907994)

Counter({'1': 8285, '0': 8099})

In [ ]:
def reverse_section(seq, length, pos):
  if length == 31 and pos == 225:
    pass
  orig_len = len(seq)
  end_idx = length + pos
  if end_idx >= len(seq):
    end_idx = end_idx % len(seq)
    prefix = seq[:end_idx % len(seq)]
    suffix = seq[pos:]
    assert len(prefix) + len(suffix) == length
    inner = seq[end_idx:pos]
    assert len(prefix) + len(suffix) + len(inner) == orig_len
    changed = list(reversed(suffix + prefix))
    out_prefix = changed[-len(prefix):] if len(prefix) else []
    out_suffix = changed[:len(suffix)] if len(suffix) else []
    out = out_prefix + inner + out_suffix
  else:
    out = seq[:pos] + list(reversed(seq[pos:pos+length])) + seq[pos+length:]
  assert len(out) == orig_len
  return out
assert reverse_section(list("21034"), 4, 3) == list("43012")
assert reverse_section(list("01234"), 3, 0) == list("21034")
assert reverse_section(list("43012"), 5, 1) == list("34210")

def knot_hash_round(seq, lengths, pos=0, skip=0):
  for length in lengths:
    try:
      seq = reverse_section(seq, length, pos)
    except:
      print(seq,length,pos)
      raise Exception
    pos = (pos + length + skip) % len(seq)
    skip += 1
  return seq,pos,skip

assert knot_hash_round(list(range(5)), [3,4,1,5])[0] == [3,4,2,1,0]

data = mapints(get_input(10, 2017)[0])
lengths = data
out,*_ = knot_hash_round(list(range(256)), lengths)
p1 = out[0]*out[1]
print("p1",p1)

def knot_hash(s):
  from functools import reduce
  from operator import xor

  seq = list(range(256))
  rounds = 64
  pos = 0
  skip = 0
  std_lengths = [17, 31, 73, 47, 23]
  lengths = [ord(ch) for ch in s] + std_lengths
  for _ in range(rounds):
    seq,pos,skip = knot_hash_round(seq, lengths, pos, skip)
  assert len(seq) == 256
  assert len(set(seq)) == len(seq)

  # reduce w/ xor
  out = [ 
    reduce( xor, seq[i*16:(i+1)*16] )
    for i in range(16)
  ]
  assert len(out) == 16
  out = list(map(lambda x: hex(x)[2:].rjust(2, '0'), out))
  return ''.join(out)

assert knot_hash("") == "a2582a3a0e66e6e86e3812dcb672a272"
assert knot_hash("AoC 2017") == "33efeb34ea91902bb2f59c9920caa6cd"
assert knot_hash("1,2,3") == "3efbe78a8d82f29979031a4aa0b16a9d"
assert knot_hash("1,2,4") == "63960835bcdc130f0b66d7ff4f6a5a8e"

p2 = knot_hash(get_input(10, 2017)[0])
assert p2 == "e0387e2ad112b7c2ef344e44885fe4d8"
print("p2",p2)



p1 11375
p2 e0387e2ad112b7c2ef344e44885fe4d8


In [ ]:
data = get_input(14, 2017)[0]
from aoc_utils import *
from collections import Counter

# simple way where we don't need to care about the actual positions
def solve1(s):
  C = Counter()
  for i in range(128):
    hash = knot_hash(f"{s}-{i}")
    for ch in hash:
      b = bin(int(ch, 16))[2:].rjust(4, '0')
      C.update(b)
  return C['1']


def solve2(s,is_p2=False):
  G = {}
  for row in range(128):
    hash = knot_hash(f"{s}-{row}")
    for colgroup in range(32):
      v = int(hash[colgroup], 16)
      for offset in range(4):
        col = offset + 4*colgroup
        G[(row,col)] = v & (1 << offset)
  if not is_p2:
    return len([v for v in G.values() if v])
  else:
    allseen = set()
    groups = 0
    for pos in G.keys():
      if not G[pos]: continue
      if pos in allseen: continue

      curgroup = set()
      newgroup = True
      search = [pos]
      while len(search):
        cur = search.pop()
        if not G[cur]: continue
        if cur in allseen: newgroup = False
        if cur in curgroup: continue
        curgroup.add(cur)
        for n in aoc_utils.neighbors4(cur):
          if G.get(n, 0):
            search.append(n)
      if newgroup: groups += 1
      allseen.update(curgroup)
    return groups



assert solve1("flqrgnkx") == solve2("flqrgnkx") == 8108
assert solve1(data) == solve2(data) == 8292
assert solve2(data, True) == 1069

print("p1",solve2(data))
print("p2",solve2(data, True))


p1 8292
p2 1069


In [ ]:
from functools import cache
data = get_input(15, 2017)
initA,initB = aoc_utils.mapints("".join(data))
testInitA = 65
testInitB = 8921
A = 16807
B = 48271
C = 2147483647

def is_multiple_of(v, multiple):
  assert multiple in [4, 8]
  if multiple == 4:
    return all(not v & 1 << x for x in [0,1])
  else:
    return all(not v & 1 << x for x in [0,1,2])

def generate(prev, factor, divider, multiple=None):
  if not multiple:
    return (prev * factor) % divider
    # return divmod(prev * factor, divider)[1]
  else:
    while True:
      prev = generate(prev, factor, divider)
      assert multiple in [4,8]
      if is_multiple_of(prev, multiple): return prev

def mkgen(start,factor,divider,multiple=None):
  v = start
  while True:
    v *= factor
    v %= divider
    if multiple is not None:
      if v % multiple == 0:
        yield v
    else:
      yield v

for i in range(1,100):
  assert is_multiple_of(i, 4) == (i % 4 == 0)
  assert is_multiple_of(i, 8) == (i % 8 == 0)

def compare(a, b):
  return a & 0xFFFF == b & 0xFFFF

assert generate(65, A, C) == 1092455
assert generate(8921, B, C) == 430625591
assert compare(245556042, 1431495498)
assert not compare(1092455, 430625591)
assert not compare(1181022009, 1233683848)
assert not compare(1744312007, 137874439)
assert not compare(1352636452, 285222916)
assert generate(65, A, C, 4) == 1352636452
assert generate(1352636452, A, C, 4) == 1992081072
assert generate(1992081072, A, C, 4) == 530830436
assert generate(1980017072, A, C, 4) == 740335192
assert generate(8921, B, C, 8) == 1233683848
assert generate(1233683848, B, C, 8) == 862516352
assert generate(862516352, B, C, 8) == 1159784568
assert generate(1159784568, B, C, 8) == 1616057672
assert generate(1616057672, B, C, 8) == 412269392

def solve(a, b, is_p2=False):
  pairs = 5_000_000 if is_p2 else 40_000_000
  out = 0
  if not is_p2:
    agen = mkgen(a, A, C)
    bgen = mkgen(b, B, C)
  else:
    agen = mkgen(a, A, C, 4)
    bgen = mkgen(b, B, C, 8)
  pct10 = pairs // 10
  for i in range(pairs):
    if i % pct10 == 0: print(f"{round(100*i/pairs)}% done, found {out}")
    a,b = next(agen),next(bgen)
    if a & 0xFFFF == b & 0xFFFF: out += 1
    # if compare(a,b): out += 1
    # if i < 5:
    #   print(i,a,b,compare(a,b))
    # if i > 5: break
  return out

print("solve p1")
p1 = solve(initA, initB, False)
print("p1", p1)
assert p1 == 573
print("solve p1")
p2 = solve(initA, initB, True)
print("p2",p2)
assert p2 == 294

aoc_utils.submit(15, 2017, 2, p2)





solve p1
0% done, found 0
10% done, found 61
20% done, found 122
30% done, found 185
40% done, found 256
50% done, found 299
60% done, found 359
70% done, found 405
80% done, found 460
90% done, found 507
p1 573
solve p1
0% done, found 0
10% done, found 42
20% done, found 63
30% done, found 98
40% done, found 120
50% done, found 153
60% done, found 175
70% done, found 203
80% done, found 228
90% done, found 259
p2 294


In [47]:
data = get_input(16, 2017)[0].split(',')
mapints = aoc_utils.mapints

from string import ascii_lowercase
x = list(ascii_lowercase[:16])

def process(x, inst):
  if inst.startswith('s'):
    dist = mapints(inst)[0]
    out = x[-dist:] + x[:len(x)-dist]
    try:
      assert len(out) == len(x)
    except:
      pass
    return out
  elif inst.startswith('x'):
    aidx,bidx = mapints(inst)
    tmpa,tmpb = x[aidx],x[bidx]
    x[aidx] = tmpb
    x[bidx] = tmpa
    return x
  elif inst.startswith('p'):
    a,b = inst[1:].split('/')
    aidx,bidx = x.index(a),x.index(b)
    return process(x, f"x{aidx}/{bidx}")

assert process(list('abcde'), 's3') == list('cdeab')
assert process(list('abcde'), 's4') == list('bcdea')
assert process(list('abcde'), 's5') == list('abcde')
assert process(list('eabcd'), 'x3/4') == list('eabdc')
assert process(list('eabcd'), 'pe/b') == list('baecd')


cat = aoc_utils.cat
def dance(x, data):
  for inst in data:
    x = process(x, inst)
  return x

def solve(data, is_p2=False):
  x = list(ascii_lowercase[:16])
  inst_idx = 0
  seen = {}
  total_steps = int(1e9) * len(data)
  for step in range(total_steps):
    if step == len(data) and not is_p2: return x
    x = process(x, data[inst_idx])
    key = (tuple(x), inst_idx)
    if key in seen:
      prev_step = seen[key]
      cycle_size = step - prev_step
      rem_steps = total_steps - step - 1
      d,m = divmod(rem_steps,cycle_size)
      if m == 0:
        return x
    else:
      seen[key] = step
    inst_idx = (inst_idx + 1) % len(data)

p1 = cat(solve(data,is_p2=False))
p2 = cat(solve(data,is_p2=True))
print(f"p1 {p1}")
print(f"p2 {p2}")
assert p1 == 'glnacbhedpfjkiom'
assert p2 == 'fmpanloehgkdcbji'

p1 glnacbhedpfjkiom
p2 fmpanloehgkdcbji
